# Projet: Fouille de données

## Réaliser par: Nouha MTIBAA-3DNI_G1

## Thème: Classification des Tweets

## Objectifs: 
* Maitriser l’API de twitter pour l’extraction des tweets
* Maitriser la partie NLP (naturallanguageprocessing) avec NLTK en Python
* Appliquer les principes de nettoyage des données
* Classer les tweets: regrouper ensemble les tweets qui sont similaires.C’est une étape qui peut être considérée comme une étape

### Importation des bibliothéques nécessaires

In [1]:
import pandas as pd 
import tweepy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [1]:
!pip install tweepy
!pip install pandas

##### On doit récupérer les Tweets à partir de Twitter en utilisant l’API de twitter et les sauvgarder dans un fichier .csv

In [ ]:
def scrape_function(words, date_since, numtweet): 
      
    # Creating DataFrame using pandas 
    db = pd.DataFrame(columns=['username', 'description', 'location', 'following', 
                               'followers', 'totaltweets', 'retweetcount', 'text', 'hashtags']) 
      
    # We are using .Cursor() to search through twitter for the required tweets. 
    # The number of tweets can be restricted using .items(number of tweets) 
    tweets = tweepy.Cursor(api.search, q=words, lang="en", 
                           since=date_since, tweet_mode='extended').items(numtweet) 
     
    # .Cursor() returns an iterable object. Each item in  
    # the iterator has various attributes that you can access to  
    # get information about each tweet 
    list_tweets = [tweet for tweet in tweets] 
      
    # Counter to maintain Tweet Count 
    i = 1  
      
    # we will iterate over each tweet in the list for extracting information about each tweet 
    for tweet in list_tweets: 
        username = tweet.user.screen_name 
        description = tweet.user.description 
        location = tweet.user.location 
        following = tweet.user.friends_count 
        followers = tweet.user.followers_count 
        totaltweets = tweet.user.statuses_count 
        retweetcount = tweet.retweet_count 
        hashtags = tweet.entities['hashtags'] 
          
        # Retweets can be distinguished by a retweeted_status attribute, 
        # in case it is an invalid reference, except block will be executed 
        try: 
            text = tweet.retweeted_status.full_text 
        except AttributeError: 
            text = tweet.full_text 
        hashtext = list() 
        for j in range(0, len(hashtags)): 
            hashtext.append(hashtags[j]['text']) 
          
        # Here we are appending all the extracted information in the DataFrame 
        ith_tweet = [username, description, location, following, 
                     followers, totaltweets, retweetcount, text, hashtext] 
        db.loc[len(db)] = ith_tweet 
          
        # Function call to print tweet data on screen 
        printtweetdata(i, ith_tweet) 
        i = i+1
    filename = 'tweets.csv'
      
    # we will save our database as a CSV file. 
    db.to_csv(filename) 
  
  
if __name__ == '__main__': 
      
    # Enter your own credentials obtained  
    # from your developer account 
    consumer_key = "ry9GMoJRUCdsHo89bkpVosu8T"
    consumer_secret = "y9Yy5X6RE3n6hEogGhVYCmGDKgDPhWuQB8sexc61xKH01Xv6MA"
    access_key = "1330861473481052162-NXO8xkGNxf0g6A7wbOjOfMPKZrfUvT"
    access_secret = "20Ye373RjvjjMfpbPHAW6kbwJURRbqRaQsnMYFBCrZpyw"
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
    auth.set_access_token(access_key, access_secret) 
    api = tweepy.API(auth,wait_on_rate_limit=True) 
      
    # Enter Hashtag and initial date 
    print("Enter Twitter HashTag to search for") 
    words = input() 
    print("Enter Date since The Tweets are required in yyyy-mm--dd") 
    date_since = input() 
      
    # number of tweets you want to extract in one run 
    numtweet =2
    scrape_function(words, date_since, numtweet) 
    print('Scraping has completed!') 

Enter Twitter HashTag to search for


##### On va afficher les dates de chaque tweet

In [2]:
# function to display data of each tweet 
def printtweetdata(n, ith_tweet): 
    print() 
    print(f"Tweet {n}:") 
    print(f"Username:{ith_tweet[0]}") 
    print(f"Description:{ith_tweet[1]}") 
    print(f"Location:{ith_tweet[2]}") 
    print(f"Following Count:{ith_tweet[3]}") 
    print(f"Follower Count:{ith_tweet[4]}") 
    print(f"Total Tweets:{ith_tweet[5]}") 
    print(f"Retweet Count:{ith_tweet[6]}") 
    print(f"Tweet Text:{ith_tweet[7]}") 
    print(f"Hashtags Used:{ith_tweet[8]}") 

##### on va lire les deux fichier dataset puis les concaténer ensemble dans un autre fichier df_final

In [4]:
df_1= pd.read_csv('tweets1.csv')
df=pd.read_csv('tweets.csv')
df_final=pd.concat([df, df_1], ignore_index=True)
print(df_final.shape)
df_final.head()

(4985, 10)


Unnamed: 0      username  \
0           0  DatGladiatah   
1           1     EliHayes_   
2           2  braincelltwo   
3           3     WuanChill   
4           4   schweitz_ay   

                                         description             location  \
0                                                NaN                  NaN   
1                             ECU Alumnus 🏴‍☠️ | 910  North Carolina, USA   
2                                     Taking it well  Art by @sloppjockey   
3  #MCFC #FlyEaglesFly. This is a Anthony Martial...                  NaN   
4                                   This guy gets it                  NaN   

   following  followers  totaltweets  retweetcount  \
0        134         14         2226             0   
1         84         76         4239             1   
2        988       4496        54644             0   
3        279        137        17911             1   
4        425         57         1027             0   

                                                text    hashtags  
0  @TulsiGabbard the biological essentialism of T...          []  
1  mighty funny how nobody in black sports media ...          []  
2  Tht time I was put in a gc full of barstool sp...          []  
3  mighty funny how nobody in black sports media ...          []  
4  @DanSoder @katienolan that Andre AD read made ...  ['sports']

In [5]:
print(df.shape)
print(df_1.shape)

(2985, 10)
(2000, 10)


### On va éliminer les colonnes dans la liste de to_drop du dataframe 

In [6]:
to_drop = ['Unnamed: 0',
           'username',
           'description',
           'location',
           'following',
           'followers',
           'totaltweets']
#Eliminer les colonnes dans la liste de to_drop du dataframe df
df.drop(to_drop, inplace=True, axis=1)
#Afficherle résultat
df.head()

retweetcount                                               text    hashtags
0             0  @TulsiGabbard the biological essentialism of T...          []
1             1  mighty funny how nobody in black sports media ...          []
2             0  Tht time I was put in a gc full of barstool sp...          []
3             1  mighty funny how nobody in black sports media ...          []
4             0  @DanSoder @katienolan that Andre AD read made ...  ['sports']

## Prétraitement

Les tweets contiennent des objets inutiles tels que des hashtags, des mentions, des liens et des signes de ponctuation qui peuvent affecter les performances d'un algorithme et doivent donc être supprimés. Tous les textes sont convertis en minuscules pour éviter que les algorithmes n'interprètent les mêmes mots avec des cas différents comme différents

In [7]:
import string , re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### On va maintenant supprimez les hashtags, les mentions et les caractères indésirables.

In [9]:
def ponctuation_sup(text_entree):
    text_entree = "".join([char for char in text_entree if char not in string.punctuation])
    text_entree = re.sub('[0-9]+', '', text_entree)
    
    return text_entree

df['Tweet_punctuation'] = df['text'].apply(lambda x: ponctuation_sup(x))
df.head(10)

retweetcount                                               text  \
0             0  @TulsiGabbard the biological essentialism of T...   
1             1  mighty funny how nobody in black sports media ...   
2             0  Tht time I was put in a gc full of barstool sp...   
3             1  mighty funny how nobody in black sports media ...   
4             0  @DanSoder @katienolan that Andre AD read made ...   
5             0  @oCHiCAx I agree. I believe in trans rights 2 ...   
6            42  constantly making fun of the wnba and women's ...   
7             1  Funny how things come full circle, @EmilyJones...   
8            42  constantly making fun of the wnba and women's ...   
9             3  Winter sports are funny, especially when you’r...   

     hashtags                                  Tweet_punctuation  
0          []  TulsiGabbard the biological essentialism of TE...  
1          []  mighty funny how nobody in black sports media ...  
2          []  Tht time I was put in a gc full of barstool sp...  
3          []  mighty funny how nobody in black sports media ...  
4  ['sports']  DanSoder katienolan that Andre AD read made me...  
5          []  oCHiCAx I agree I believe in trans rights  but...  
6          []  constantly making fun of the wnba and womens s...  
7          []  Funny how things come full circle EmilyJonesMc...  
8          []  constantly making fun of the wnba and womens s...  
9          []  Winter sports are funny especially when you’re...

### Tokenisation, lemmatisation et suppression des mots vides

#### La tokenisation cherche à transformer un texte en une série de tokens individuels. Dans l’idée, chaque token représente un mot, et identifier des mots semble être une tâche relativement simple.

In [10]:

df['Tweet_no_mot'] = df['Tweet_punctuation'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
def tokenization(text):
    text = re.split(' ', text)
    return text

df['Tweet_tokenized'] = df['Tweet_no_mot'].apply(lambda x: tokenization(x.lower()))

df.head()


retweetcount                                               text  \
0             0  @TulsiGabbard the biological essentialism of T...   
1             1  mighty funny how nobody in black sports media ...   
2             0  Tht time I was put in a gc full of barstool sp...   
3             1  mighty funny how nobody in black sports media ...   
4             0  @DanSoder @katienolan that Andre AD read made ...   

     hashtags                                  Tweet_punctuation  \
0          []  TulsiGabbard the biological essentialism of TE...   
1          []  mighty funny how nobody in black sports media ...   
2          []  Tht time I was put in a gc full of barstool sp...   
3          []  mighty funny how nobody in black sports media ...   
4  ['sports']  DanSoder katienolan that Andre AD read made me...   

                                        Tweet_no_mot  \
0  TulsiGabbard biological essentialism TERFs fun...   
1  mighty funny nobody black sports media standin...   
2  time full barstool sports type funny accounts ...   
3  mighty funny nobody black sports media standin...   
4  DanSoder katienolan that Andre read made swerv...   

                                     Tweet_tokenized  
0  [tulsigabbard, biological, essentialism, terfs...  
1  [mighty, funny, nobody, black, sports, media, ...  
2  [time, full, barstool, sports, type, funny, ac...  
3  [mighty, funny, nobody, black, sports, media, ...  
4  [dansoder, katienolan, that, andre, read, made...

In [11]:
df['Tweet_tokenized'] = df['Tweet_no_mot'].apply(lambda x: tokenization(x.lower()))

df.head()

retweetcount                                               text  \
0             0  @TulsiGabbard the biological essentialism of T...   
1             1  mighty funny how nobody in black sports media ...   
2             0  Tht time I was put in a gc full of barstool sp...   
3             1  mighty funny how nobody in black sports media ...   
4             0  @DanSoder @katienolan that Andre AD read made ...   

     hashtags                                  Tweet_punctuation  \
0          []  TulsiGabbard the biological essentialism of TE...   
1          []  mighty funny how nobody in black sports media ...   
2          []  Tht time I was put in a gc full of barstool sp...   
3          []  mighty funny how nobody in black sports media ...   
4  ['sports']  DanSoder katienolan that Andre AD read made me...   

                                        Tweet_no_mot  \
0  TulsiGabbard biological essentialism TERFs fun...   
1  mighty funny nobody black sports media standin...   
2  time full barstool sports type funny accounts ...   
3  mighty funny nobody black sports media standin...   
4  DanSoder katienolan that Andre read made swerv...   

                                     Tweet_tokenized  
0  [tulsigabbard, biological, essentialism, terfs...  
1  [mighty, funny, nobody, black, sports, media, ...  
2  [time, full, barstool, sports, type, funny, ac...  
3  [mighty, funny, nobody, black, sports, media, ...  
4  [dansoder, katienolan, that, andre, read, made...

In [12]:
df.head()

retweetcount                                               text  \
0             0  @TulsiGabbard the biological essentialism of T...   
1             1  mighty funny how nobody in black sports media ...   
2             0  Tht time I was put in a gc full of barstool sp...   
3             1  mighty funny how nobody in black sports media ...   
4             0  @DanSoder @katienolan that Andre AD read made ...   

     hashtags                                  Tweet_punctuation  \
0          []  TulsiGabbard the biological essentialism of TE...   
1          []  mighty funny how nobody in black sports media ...   
2          []  Tht time I was put in a gc full of barstool sp...   
3          []  mighty funny how nobody in black sports media ...   
4  ['sports']  DanSoder katienolan that Andre AD read made me...   

                                        Tweet_no_mot  \
0  TulsiGabbard biological essentialism TERFs fun...   
1  mighty funny nobody black sports media standin...   
2  time full barstool sports type funny accounts ...   
3  mighty funny nobody black sports media standin...   
4  DanSoder katienolan that Andre read made swerv...   

                                     Tweet_tokenized  
0  [tulsigabbard, biological, essentialism, terfs...  
1  [mighty, funny, nobody, black, sports, media, ...  
2  [time, full, barstool, sports, type, funny, ac...  
3  [mighty, funny, nobody, black, sports, media, ...  
4  [dansoder, katienolan, that, andre, read, made...

On vas utiliser la bibliothèque NLTK pour effectuer une analyse de chaque tweet et le transformer en un ensemble de mots en suivant les différentes étapes de base du processus NLP (Natural Language Processing)

In [12]:
!pip install nltk

In [13]:

import nltk
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')
print(stopword)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nouha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
df['Tweet_words'] = df['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))
df.head(10)

retweetcount                                               text  \
0             0  @TulsiGabbard the biological essentialism of T...   
1             1  mighty funny how nobody in black sports media ...   
2             0  Tht time I was put in a gc full of barstool sp...   
3             1  mighty funny how nobody in black sports media ...   
4             0  @DanSoder @katienolan that Andre AD read made ...   
5             0  @oCHiCAx I agree. I believe in trans rights 2 ...   
6            42  constantly making fun of the wnba and women's ...   
7             1  Funny how things come full circle, @EmilyJones...   
8            42  constantly making fun of the wnba and women's ...   
9             3  Winter sports are funny, especially when you’r...   

     hashtags                                  Tweet_punctuation  \
0          []  TulsiGabbard the biological essentialism of TE...   
1          []  mighty funny how nobody in black sports media ...   
2          []  Tht time I was put in a gc full of barstool sp...   
3          []  mighty funny how nobody in black sports media ...   
4  ['sports']  DanSoder katienolan that Andre AD read made me...   
5          []  oCHiCAx I agree I believe in trans rights  but...   
6          []  constantly making fun of the wnba and womens s...   
7          []  Funny how things come full circle EmilyJonesMc...   
8          []  constantly making fun of the wnba and womens s...   
9          []  Winter sports are funny especially when you’re...   

                                        Tweet_no_mot  \
0  TulsiGabbard biological essentialism TERFs fun...   
1  mighty funny nobody black sports media standin...   
2  time full barstool sports type funny accounts ...   
3  mighty funny nobody black sports media standin...   
4  DanSoder katienolan that Andre read made swerv...   
5  oCHiCAx agree believe trans rights genetically...   
6    constantly making wnba womens sports isnt funny   
7  Funny things come full circle EmilyJonesMcCoy ...   
8    constantly making wnba womens sports isnt funny   
9  Winter sports funny especially when you’re get...   

                                     Tweet_tokenized  \
0  [tulsigabbard, biological, essentialism, terfs...   
1  [mighty, funny, nobody, black, sports, media, ...   
2  [time, full, barstool, sports, type, funny, ac...   
3  [mighty, funny, nobody, black, sports, media, ...   
4  [dansoder, katienolan, that, andre, read, made...   
5  [ochicax, agree, believe, trans, rights, genet...   
6  [constantly, making, wnba, womens, sports, isn...   
7  [funny, things, come, full, circle, emilyjones...   
8  [constantly, making, wnba, womens, sports, isn...   
9  [winter, sports, funny, especially, when, you’...   

                                         Tweet_words  
0  [tulsigabbard, biological, essentialism, terfs...  
1  [mighty, funny, nobody, black, sports, media, ...  
2  [time, full, barstool, sports, type, funny, ac...  
3  [mighty, funny, nobody, black, sports, media, ...  
4  [dansoder, katienolan, andre, read, made, swer...  
5  [ochicax, agree, believe, trans, rights, genet...  
6  [constantly, making, wnba, womens, sports, isn...  
7  [funny, things, come, full, circle, emilyjones...  
8  [constantly, making, wnba, womens, sports, isn...  
9  [winter, sports, funny, especially, you’re, ge...

Le stemming consiste à réduire un mot dans sa forme « racine ». Le but du stemming est de regrouper de nombreuses variantes d’un mot comme un seul et même mot. Par exemple, une fois que l’on applique un stemming sur « Chiens » ou « Chien », le mot résultant est le même. 

In [17]:
ps = nltk.PorterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

df['Tweet_stemmed'] = df['Tweet_words'].apply(lambda x: stemming(x))
df.head()

retweetcount                                               text  \
0             0  @TulsiGabbard the biological essentialism of T...   
1             1  mighty funny how nobody in black sports media ...   
2             0  Tht time I was put in a gc full of barstool sp...   
3             1  mighty funny how nobody in black sports media ...   
4             0  @DanSoder @katienolan that Andre AD read made ...   

     hashtags                                  Tweet_punctuation  \
0          []  TulsiGabbard the biological essentialism of TE...   
1          []  mighty funny how nobody in black sports media ...   
2          []  Tht time I was put in a gc full of barstool sp...   
3          []  mighty funny how nobody in black sports media ...   
4  ['sports']  DanSoder katienolan that Andre AD read made me...   

                                        Tweet_no_mot  \
0  TulsiGabbard biological essentialism TERFs fun...   
1  mighty funny nobody black sports media standin...   
2  time full barstool sports type funny accounts ...   
3  mighty funny nobody black sports media standin...   
4  DanSoder katienolan that Andre read made swerv...   

                                     Tweet_tokenized  \
0  [tulsigabbard, biological, essentialism, terfs...   
1  [mighty, funny, nobody, black, sports, media, ...   
2  [time, full, barstool, sports, type, funny, ac...   
3  [mighty, funny, nobody, black, sports, media, ...   
4  [dansoder, katienolan, that, andre, read, made...   

                                         Tweet_words  \
0  [tulsigabbard, biological, essentialism, terfs...   
1  [mighty, funny, nobody, black, sports, media, ...   
2  [time, full, barstool, sports, type, funny, ac...   
3  [mighty, funny, nobody, black, sports, media, ...   
4  [dansoder, katienolan, andre, read, made, swer...   

                                       Tweet_stemmed  
0  [tulsigabbard, biolog, essenti, terf, funni, a...  
1  [mighti, funni, nobodi, black, sport, media, s...  
2  [time, full, barstool, sport, type, funni, acc...  
3  [mighti, funni, nobodi, black, sport, media, s...  
4  [dansod, katienolan, andr, read, made, swerv, ...

La stemming et la lemmatisation sont des techniques de normalisation de texte (ou parfois appelées normalisation de mots ) dans le domaine du traitement du langage naturel qui sont utilisées pour préparer du texte, des mots et des documents pour un traitement ultérieur.

In [18]:
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

df['Tweet_lemmatized'] = df['Tweet_words'].apply(lambda x: lemmatizer(x))
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nouha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


retweetcount                                               text  \
0             0  @TulsiGabbard the biological essentialism of T...   
1             1  mighty funny how nobody in black sports media ...   
2             0  Tht time I was put in a gc full of barstool sp...   
3             1  mighty funny how nobody in black sports media ...   
4             0  @DanSoder @katienolan that Andre AD read made ...   

     hashtags                                  Tweet_punctuation  \
0          []  TulsiGabbard the biological essentialism of TE...   
1          []  mighty funny how nobody in black sports media ...   
2          []  Tht time I was put in a gc full of barstool sp...   
3          []  mighty funny how nobody in black sports media ...   
4  ['sports']  DanSoder katienolan that Andre AD read made me...   

                                        Tweet_no_mot  \
0  TulsiGabbard biological essentialism TERFs fun...   
1  mighty funny nobody black sports media standin...   
2  time full barstool sports type funny accounts ...   
3  mighty funny nobody black sports media standin...   
4  DanSoder katienolan that Andre read made swerv...   

                                     Tweet_tokenized  \
0  [tulsigabbard, biological, essentialism, terfs...   
1  [mighty, funny, nobody, black, sports, media, ...   
2  [time, full, barstool, sports, type, funny, ac...   
3  [mighty, funny, nobody, black, sports, media, ...   
4  [dansoder, katienolan, that, andre, read, made...   

                                         Tweet_words  \
0  [tulsigabbard, biological, essentialism, terfs...   
1  [mighty, funny, nobody, black, sports, media, ...   
2  [time, full, barstool, sports, type, funny, ac...   
3  [mighty, funny, nobody, black, sports, media, ...   
4  [dansoder, katienolan, andre, read, made, swer...   

                                       Tweet_stemmed  \
0  [tulsigabbard, biolog, essenti, terf, funni, a...   
1  [mighti, funni, nobodi, black, sport, media, s...   
2  [time, full, barstool, sport, type, funni, acc...   
3  [mighti, funni, nobodi, black, sport, media, s...   
4  [dansod, katienolan, andr, read, made, swerv, ...   

                                    Tweet_lemmatized  
0  [tulsigabbard, biological, essentialism, terfs...  
1  [mighty, funny, nobody, black, sport, medium, ...  
2  [time, full, barstool, sport, type, funny, acc...  
3  [mighty, funny, nobody, black, sport, medium, ...  
4  [dansoder, katienolan, andre, read, made, swer...

In [22]:
df.Tweet_lemmatized.to_csv('Tweets_cleaned.csv',index = False)

C:\Users\nouha\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [24]:
new_tweet_df= pd.read_csv('Tweets_cleaned.csv')
new_tweet_df.head(10)


['tulsigabbard', 'biological', 'essentialism', 'terfs', 'funny', 'apparently', 'vouch', 'equality', 'insist', 'moment', 'time', 'youre', 'inferior', 'unable', 'compete', 'biological', 'maybe', 'youre', 'good', 'sport']
0  ['mighty', 'funny', 'nobody', 'black', 'sport'...                                                                                                                                                                        
1  ['time', 'full', 'barstool', 'sport', 'type', ...                                                                                                                                                                        
2  ['mighty', 'funny', 'nobody', 'black', 'sport'...                                                                                                                                                                        
3  ['dansoder', 'katienolan', 'andre', 'read', 'm...                                                                                                                                                                        
4  ['ochicax', 'agree', 'believe', 'trans', 'righ...                                                                                                                                                                        
5  ['constantly', 'making', 'wnba', 'woman', 'spo...                                                                                                                                                                        
6  ['funny', 'thing', 'come', 'full', 'circle', '...                                                                                                                                                                        
7  ['constantly', 'making', 'wnba', 'woman', 'spo...                                                                                                                                                                        
8  ['winter', 'sport', 'funny', 'especially', 'yo...                                                                                                                                                                        
9  ['funny', 'thing', 'come', 'full', 'circle', '...

In [25]:
print('Dataset size:',new_tweet_df.shape)
print('Columns are:',new_tweet_df.columns)

Dataset size: (2984, 1)
Columns are: Index(['['tulsigabbard', 'biological', 'essentialism', 'terfs', 'funny', 'apparently', 'vouch', 'equality', 'insist', 'moment', 'time', 'youre', 'inferior', 'unable', 'compete', 'biological', 'maybe', 'youre', 'good', 'sport']'], dtype='object')


## Vectorisation
- Les données nettoyées en une seule ligne en passant new_tweet_df dans le CountVectorizer

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X=cv.fit_transform(new_tweet_df.Tweet_lemmatized)
print(X)

AttributeError: 'DataFrame' object has no attribute 'Tweet_lemmatized'

## Classification des tweets

* Cette approche utilise la technique de création d'un ensemble de mots qui peuvent être classés en toute confiance comme  appartenant à une catégorie particulière .
* On va Utiliser l’algorithme K-Means pour classer les Tweets en 30 classes.

In [21]:
from sklearn.cluster import KMeans
wcss=[]
for i in range(3,30):
   
    Kmeans=KMeans(n_clusters=i,init='k-means++',max_iter=300,n_init=10,random_state=0,verbose=True)
    Kmeans.fit(X)
    wcss.append(Kmeans.inertia_)

 ********** Nombre de cluster : ************ 3


NameError: name 'X' is not defined

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(3,30),wcss)
plt.xlabel('number of clusters')
plt.ylabel('word per cluster')
plt.show()

In [ ]:
true_k=30
Kmeans=KMeans(n_clusters=true_k,init='k-means++',n_init=1)
Kmeans.fit(X)
print("Top terms per cluster:")
order_centroids = Kmeans.cluster_centers_.argsort()[:, ::-1]
terms = cv.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i)
    print("-----------------------")
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind])
    print()
print("\n")

In [ ]:
i=0
j=0
while i<28:
    while True: 
        Y=cv.transform([new_tweet_df.Tweet_lemmatized[j]])
        prediction=Kmeans.predict(Y)
        if i == prediction:
            print("Tweet of cluster "+str(prediction)+" : "+df.Tweet_lemmatized[i])
            print ("-----------------------------------------------")
            print("\n")
            j=0
            break
        j+=1
    i+=1